# Sección 0: Demostración Web Scraping

El objetivo del proyecto era extraer la información de gerentes históricos y actuales de muchas compañías. Se accedió al portal de Supercias. Extraer la información manualmente tomaba, en promedio, un minuto por observación. Adicionalmente, después de extraer muchas observaciones, el proceso se volvía fatigante y solía haber errores en el guardado de la información. Se optó por utilizar un script de web scraping para extraer la información automáticamente y sin ningún tipo de error humano.

In [5]:
import pandas as pd
# #Leer el archivo con los expedientes de las compañías de las que se quería conocer los gerentes
# #actuales
# import cv2
# import pytesseract
# import os
# from bs4 import BeautifulSoup
# import html5lib
# from selenium.webdriver.support.ui import Select
# import pandas as pd
dframe = pd.read_csv(r"..\exps.txt",encoding = "latin-1",sep="\t")
exps = list(set(dframe["EXPEDIENTE"]))[:1000]
len(exps)

1000

In [3]:
#Definición de la función que extrae y guarda la información.
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException,TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,NoSuchElementException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import pandas as pd
import cv2
import pytesseract
import os
import urllib.request as ur
import html5lib
from bs4 import BeautifulSoup
import os
from urllib.error import HTTPError

def solve(src):
    #Save and Read the captcha image
    ur.urlretrieve(src,"captcha.png")
    #Gaming PC
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    #DHUB PC
#    pytesseract.pytesseract.tesseract_cmd = r"C:\Users\eduranh\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
    #Other PC
#     pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Py Tesseract\tesseract.exe"
    img = cv2.imread("captcha.png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    answer = pytesseract.image_to_string(img)
#     print(answer)
    #Remove image after reading it so the next iteration can save a new captcha.
    os.remove("captcha.png")
    return answer
    #Give a response to the captcha
def checkDuplicates(driver):
    path = r"C:\Users\EDWIN\OneDrive - Universidad San Francisco de Quito\USFQ Online\Décimosegundo Semestre\Web Scraping Course\Financials"
    files = os.listdir(path)
    for file in files:
        check = "(" in file
        if check == True:
            print(file)
            os.remove(f"{path}\\{file}")
            return check
    
def solve_other(driver):
    answer = 0
    time.sleep(3)
    try:
        captcha = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"frmCaptcha:captchaImage")))
        src = captcha.get_attribute("src")
        answer = solve(src)
        bar = driver.find_element(By.ID,"frmCaptcha:captcha")
        time.sleep(1)
        bar.send_keys(answer)
        bar.send_keys(Keys.ENTER)
        print("Captcha Solved")
        return answer
    except:
#         print("No Captcha")
        pass
    return answer
def lookForAll(exps):
    path = r"C:\Users\EDWIN\OneDrive - Universidad San Francisco de Quito\USFQ Online\Décimosegundo Semestre\Web Scraping Course"
    url = "https://appscvsconsultas.supercias.gob.ec/consultaCompanias/societario/busquedaCompanias.jsf"
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs",{"download.default_directory":path,
                                          "download.prompt_for_download":False,
                                          "download.directory_upgrade":True,
                                          "plugins.always_open_pdf_externally":True})
    driver = webdriver.Chrome(options=options)

    used_rucs = []
    count = 0
    #Empty dataframe (where all the other dataframes will be stored):
    frame = pd.DataFrame()
    while len(exps) > 0:
#         try:
        time.sleep(0.5)
        count += 1
        driver.get(url)
        used = exps.pop(0)
        expedient = driver.find_elements(By.CLASS_NAME,
                                         "ui-autocomplete-input.ui-inputfield.ui-widget.ui-state-default.ui-corner-all"
                                        )
        expedient[0].send_keys(
            str(used)
        )
        time.sleep(1)
        expedient[0].send_keys(Keys.ENTER)
        time.sleep(1)
        #Find the Capthca image
        captcha = driver.find_element(By.ID,"frmBusquedaCompanias:captchaImage")
        src = captcha.get_attribute("src")
        answer = solve(src)

        capi = driver.find_element(By.CSS_SELECTOR,
            "input.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")
        capi.send_keys(answer)
        time.sleep(0.5)

        #Click Yearly Information Tab:
        btn = driver.find_element(By.ID,"frmMenu:menuInformacionAnualPresentada")
        btn.click()
        #Find the Historical Balance sheets submitted by the company
        box = driver.find_element(By.ID,"frmInformacionCompanias:tblInformacionAnual:j_idt587:filter")
        box.send_keys("3.1.1")

        #In case captcha comes up, solve it:
        solve_other(driver)
        time.sleep(1.5)
        elements = Select(driver.find_element(By.CSS_SELECTOR,
                               "select.ui-paginator-rpp-options.ui-widget.ui-state-default.ui-corner-left"))
        elements.select_by_visible_text("*")

        #Save page source to find stuff with BS4:
        source2 = driver.page_source

        #Use stored page_source1 to find the pertinent table:
        soup = BeautifulSoup(source1,"lxml")
        #Isolate the table we want to grab information from:
        table = soup.find("div",class_="ui-datatable-tablewrapper")
        #Convert this table to a dataframe:
        current = pd.read_html(str(table))[0]
        current.insert(0,"Expediente",used)

        #Use the second stored page_source to find the pertinent table
        soup2 = BeautifulSoup(source2,"lxml")
        #Grab the table and save it as dataframe.
        table2 = soup2.find("div",class_ = "ui-datatable-tablewrapper")
        previous = pd.read_html(str(table2))[0]
        #Clear away the Filter by from the column names:
        for i in previous:
            if "Filter by" in i:
                word = i.split(" ")[-1]
                previous.rename(columns = {i:i.replace(f"Filter by {word}","")},inplace = True)
        previous.insert(0,"Expediente",used)

        #Join the Created Dataframes
        frame = pd.concat([frame,current,previous]).reset_index(drop = True)
#         except:
#             exps.insert(0,used)
#             continue
        driver.execute_script("window.open('');")
        driver.switch_to.window(driver.window_handles[-1])
    display(frame)
    return driver,frame

# Placeholder: Repurposing of the scraping script

### Part 1: Navigation and Download Functions.

The navFin function records all the navigation steps until we can interact with the PDFs of the financial information. We express the thing as a function, because we might need to reutilize it if a PDF doesn't load properly. Plus it makes the final script look cleaner. We had to make changes for it to navigate to the general documents provided by the company, as this is the place where the audited financial statements are.

The pdfShow function was refurbished to look for the audited financial statements for each company, and to switch to each individual financial statement if the companies do not have the audited version. This had some issues, which is the refresh rate of the information table. If the server is performing well, this takes milliseconds; however, when it's underperforming, the table might not refresh at all. This is quite an issue, and there is no way to make the script wait a specific time for it to update.

The download method concatenates these and makes the necessary clicks (by offset) to download the items. Since the items have the same class as the original version, there shouldn't be much of a need to switch it much. The only change to be done is to make a parent element find the children that have the same class name as the balance sheets that were looked for before.

In [238]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException,TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,NoSuchElementException,UnexpectedAlertPresentException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.select import Select
import pandas as pd
import cv2
import pytesseract
import os
import urllib.request as ur
import html5lib
from bs4 import BeautifulSoup
import os
from urllib.error import HTTPError

#Create a custom error so I can use it whenever I want the program to fail
class PlannedException(Exception):
    "Exception for whenever I need a part of the program to fail"
    pass

class NoAuditedException(Exception):
    "Exception for when no audited financials are available"
    pass

def update():
    content = list(pd.read_csv(r"..\exps.txt",sep = "\t",encoding = "latin-1")["EXPEDIENTE"])
    a = os.listdir("Financials")
    for i in a:
        content.remove(int(i))
    return content

def solve(src):
    #Save and Read the captcha image
    ur.urlretrieve(src,"captcha.png")
    #Gaming PC
    # pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    #DHUB PC
    pytesseract.pytesseract.tesseract_cmd = r"C:\Users\eduranh\AppData\Local\Programs\Tesseract-OCR\tesseract.exe"
    #Other PC
#     pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Py Tesseract\tesseract.exe"
    img = cv2.imread("captcha.png")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    answer = pytesseract.image_to_string(img)
#     print(answer)
    #Remove image after reading it so the next iteration can save a new captcha.
    os.remove("captcha.png")
    return answer

def solve_other(driver):
    answer = 0
    time.sleep(3)
    try:
        captcha = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.ID,"frmCaptcha:captchaImage")))
        src = captcha.get_attribute("src")
        answer = solve(src)
        bar = driver.find_element(By.ID,"frmCaptcha:captcha")
        time.sleep(1)
        bar.send_keys(answer)
        bar.send_keys(Keys.ENTER)
        print("Captcha Solved")
        return answer
    except:
#         print("No Captcha")
        pass
    return answer

def navFin(driver,fails = 0):
    try:
        #Click Yearly Information Tab:
        btn = WebDriverWait(driver,10).until(
            EC.presence_of_element_located((
                By.ID,"frmMenu:menuDocumentacion")))
        btn.click()

        #In case captcha comes up, solve it:
        time.sleep(1)
        solve_other(driver)
        time.sleep(1)

        #Open the Economic Documents tab
        driver.find_elements(By.CLASS_NAME,"ui-tabs-header.ui-state-default.ui-corner-top")[2].click()
  
  #In case captcha comes up, solve it:
        time.sleep(1)
        solve_other(driver)
        time.sleep(1)

        print("Financial Info Opened Successfully")
        fails = 0
    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError,NoSuchElementException):
        print("Nav. to financial failed")
        fails += 1
        driver.refresh()
        if fails < 10:
            print(f"Fail number {fails}")
            navFin(driver,fails)
        else:
            print("Too many failures")
            raise PlannedException("Too many failures.")
            
def auditedShow(driver):
    time.sleep(1)
    wait = WebDriverWait(driver,10,0.2)
    wait.until(EC.visibility_of_element_located((By.ID,"frmInformacionCompanias:tabViewDocumentacion:tblDocumentosEconomicos")))
    #Find the adequate information table.
    m_table = driver.find_element(By.ID,"frmInformacionCompanias:tabViewDocumentacion:tblDocumentosEconomicos")
    #Find the pertinent search bar with the proper search criteria.
    s_bar = m_table.find_element(By.CLASS_NAME,"ui-column-filter.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")
    s_bar.send_keys("AUD")
    #Display every single document possible
    elements = Select(m_table.find_element(By.CSS_SELECTOR,
                        "select.ui-paginator-rpp-options.ui-widget.ui-state-default.ui-corner-left"))
    elements.select_by_visible_text("*")
    #Audited Financials won't always be available. If it's the case, then extract every statement separately.
    #Wait until the table updates to check if there are any elements to download.
    time.sleep(1)
    wait.until(EC.element_to_be_clickable(s_bar))
    m_table.click()
 
    if len(m_table.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-even")) == 0:
        print("No audited financial statements found. Resorting to downloading the regular financials")
        s_bar.send_keys(Keys.CONTROL + "a")
        s_bar.send_keys(Keys.DELETE)
        time.sleep(0.5)
        wait.until(EC.element_to_be_clickable(s_bar))
        raise NoAuditedException
    time.sleep(3)
    
def finShow(driver,crit):
    print(crit)
    time.sleep(5)
    wait = WebDriverWait(driver,10,0.2)
    wait.until(EC.element_to_be_clickable((By.ID,"frmInformacionCompanias:tabViewDocumentacion:tblDocumentosEconomicos")))
    #Find the adequate information table.
    m_table = driver.find_element(By.ID,"frmInformacionCompanias:tabViewDocumentacion:tblDocumentosEconomicos")
    #Find the pertinent search bar with the proper search criteria.
    s_bar = m_table.find_element(By.CLASS_NAME,"ui-column-filter.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")
    s_bar.send_keys(Keys.CONTROL + "a")
    s_bar.send_keys(Keys.DELETE)
    time.sleep(5)
    s_bar.send_keys(crit)
    
    #Solve Captcha if it pops up
    time.sleep(0.5)
    solve_other(driver)
    time.sleep(0.5)

    #Display every single document possible
    elements = Select(m_table.find_element(By.CSS_SELECTOR,
                        "select.ui-paginator-rpp-options.ui-widget.ui-state-default.ui-corner-left"))
    elements.select_by_visible_text("*")

    #Solve Captcha if it pops up
    time.sleep(0.5)
    solve_other(driver)
    time.sleep(0.5)
 
def join(rows_u):
    d = []
    for i in rows_u:
        d.extend(i)
    return d

def auditedDown(driver):
    try:
        wait = WebDriverWait(driver,10)
        print("Beginning Navigation")
        navFin(driver)
        time.sleep(3)
        auditedShow(driver)
        print("Navigation Finished")
        time.sleep(1)
    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError) as e:
        print("Load Failed...")
        driver.refresh()
        download(driver,p,count)
    except NoSuchElementException:
        print("No PDF available for this year")
        count += 1
        download(driver,p,count)
    except IndexError:
        print("No more PDFs")

def separator(crit,p):
    if not os.path.isdir(f"{p}\\{crit}"):
        with open(f"{crit}","w") as f:
            f.write(" ")

def download(driver,p,count = 0,fails = 0,audit = True, crit = "AUD"):
    a = audit
    print(f"Audit = {a}")
    try:
        #Fake rows element to start while loop
        rows = range(100)
        #Set an action object to execute clicks on the webpage at will.
        action = ActionChains(driver)
        #Loop until every single file has been downloaded.
        while len(rows[count:]) > 0:
            file_num = len(os.listdir(p))
            print(f"Current amount of files in dir: {file_num}")
            print(f"File number {count}")
            
            #Rows where the PDFs are:
            wait.until(EC.element_to_be_clickable((By.ID,"frmInformacionCompanias:tabViewDocumentacion:tblDocumentosEconomicos")))
            m_table = driver.find_element(By.ID,"frmInformacionCompanias:tabViewDocumentacion:tblDocumentosEconomicos")
            rows_p = m_table.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-even")
            rows_i = m_table.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-odd")
            rows_u = zip(rows_p,rows_i)
            rows = join(rows_u)

            #Are there any documents?
            if len(m_table.find_elements(By.CLASS_NAME,"ui-widget-content.ui-datatable-even")) == 0 and audit == True:
                print("No audited financials")
                raise NoAuditedException
            separator(crit,p)
            #Open the desired PDF.
    #         wait.until(EC.element_to_be_clickable((By.CLASS_NAME,"ui-widget-content.ui-datatable-even")))
            pdf = rows[count].find_element(By.CLASS_NAME,"ui-commandlink.ui-widget")
            driver.execute_script("arguments[0].click();",pdf)
            #Solve Captha if it pops up
            time.sleep(0.5)
            solve_other(driver)
            time.sleep(2)
            print("Starting Download")
            #Download the files.
            wait.until(EC.element_to_be_clickable((By.ID,"dlgPresentarDocumentoPdf")))
            x = driver.find_element(By.ID,"dlgPresentarDocumentoPdf")
            if count == 0:
                print("First Download")
        #         Download Button (First Download)
                action.move_to_element_with_offset(x,360,-225)
                action.click()
                action.perform()
                time.sleep(3)
            else:
                #Download Button (After first download)
                action.move_to_element_with_offset(x,365,-210)
                action.click()
                action.perform()
           #X Button (After first download)
            action.move_to_element_with_offset(x,450,-240)
            action.click()
            action.perform()
            #Check if file was actually downloaded
            print(f"New amount of files in dir: {len(os.listdir(p))}")
            if file_num == len(os.listdir(p)):
                if fails > 10:
                    print("Documents consistently not downloading. Starting again.")
                    raise PlannedException("Documents consistently not downloading. Starting again.")
                driver.refresh()
                fails += 1
                
                print("No file was downloaded. Repeating iteration...")
                navFin(driver)
                time.sleep(1)
                #Download the right kind of documents.
                if audit == True:
                    auditedShow(driver)
                else:
                    finShow(driver,crit)
                download(driver = driver,p = p,
                        count = count,fails = fails,audit = a,crit = crit)
            count += 1      
    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError) as e:
        print("Load Failed...")
        driver.refresh()
        navFin(driver)
        time.sleep(3)
        #Download the right kind of documents.
        if audit == True:
            auditedShow(driver)
        else:
            finShow(driver,crit)
        download(driver = driver,p = p,
                 count = count,fails = fails,audit = a,crit = crit)
    except NoSuchElementException:
        print("No PDF available for this year")
        count += 1
        print("Load Failed...")
        driver.refresh()
        navFin(driver)
        time.sleep(3)
        if audit == True:
            print("Audit is equal to true")
            auditedShow(driver)
        else:
            print(f"Here, Audit is equal to:{audit}")
            finShow(driver,crit)
            download(driver = driver,
                     p = p,count = count,fails = fails,audit = a,crit = crit)

    except IndexError:
        print("No more PDFs")

In [65]:
# navFin(driver)
# finShow(driver,"situ")
path = f"{os.getcwd()}\\Financials\\47845"
download(driver,path)

Current amount of files in dir: 0
File number 0
Load Failed...
Current amount of files in dir: 0
File number 0
Load Failed...
Current amount of files in dir: 0
File number 0
Load Failed...
Current amount of files in dir: 0
File number 0
Load Failed...
Current amount of files in dir: 0
File number 0
Load Failed...
Current amount of files in dir: 0
File number 0
Load Failed...


KeyboardInterrupt: 

### Extraction Script

In [239]:
#Make master directory to download the files:
if not os.path.isdir("Financials"):
    os.mkdir("Financials")

#Get the absolute path for the master financials directory
path = f"{os.getcwd()}\\Financials"
#Define URL for the info portal
url = "https://appscvsconsultas.supercias.gob.ec/consultaCompanias/societario/busquedaCompanias.jsf"

#Read file with company exps
dframe = list(pd.read_csv(r"..\exps.txt",sep = "\t",encoding = "latin-1")["EXPEDIENTE"])
exps = update()[:1000]
len(exps)

used_rucs = []

while len(exps) != 0:
    used = exps.pop(0)
    #Generate directory for the company's information
    directory = f"Financials\\{used}"
    if not os.path.isdir(directory):
        os.mkdir(directory)       
    options = webdriver.ChromeOptions()
    options.add_experimental_option("prefs",{"download.default_directory":f"{path}\\{used}",
                                      "download.prompt_for_download":False,
                                      "download.directory_upgrade":True,
                                      "plugins.always_open_pdf_externally":True})
    driver = webdriver.Chrome(options=options)
    driver.set_window_size(1004,708)
    try:
        driver.get(url)
        wait = WebDriverWait(driver, 3)
        expedient = wait.until(EC.presence_of_element_located((By.ID,
                                         "frmBusquedaCompanias:parametroBusqueda_input")))
        expedient.send_keys(str(used))
        time.sleep(2)
        expedient.send_keys(Keys.ENTER)
        #Find the Capthca image
        wait = WebDriverWait(driver,3) #Wait element
        captcha = wait.until(EC.presence_of_element_located((By.ID,"frmBusquedaCompanias:captchaImage")))
        src = captcha.get_attribute("src")
        answer = solve(src)

        capi = driver.find_element(By.CSS_SELECTOR,
            "input.ui-inputfield.ui-inputtext.ui-widget.ui-state-default.ui-corner-all")
        capi.send_keys(answer)
        #Continue Button Click
        driver.find_element(By.CLASS_NAME,"ui-button-text.ui-c")
        time.sleep(1)
        
        #Navigate to the financial information we seek.
        navFin(driver)
        time.sleep(3)
        auditedShow(driver)
        break
        download(driver,p = directory)

    except (NoAuditedException):
        fins = ["balance","resultado","flujos"]
        for criter in fins:
            finShow(driver,criter)
            time.sleep(3)
            download(driver,p,audit=False,crit = criter)

    except (TimeoutException,ElementNotInteractableException,ElementClickInterceptedException,HTTPError,NoSuchElementException,StaleElementReferenceException) as e:
        print("Something Went Wrong" + str(e))
        exps.insert(0,used)
        continue
    except (RecursionError,UnexpectedAlertPresentException,PlannedException):
        print("Maximum Recursion exceeded")
        exps.insert(0,used)
        files = os.listdir(f"Financials\\{used}")
        for file in files:
            os.remove(f"Financials\\{used}\\{file}")
    except (KeyboardInterrupt):
        print("Stopping program...")
        files = os.listdir(f"Financials\\{used}")
        for file in files:
            os.remove(f"Financials\\{used}\\{file}")     
        os.rmdir(f"Financials\\{used}") 
        raise KeyboardInterrupt
# driver.close()

Nav. to financial failed
Fail number 1
Nav. to financial failed
Fail number 2
Captcha Solved
Captcha Solved
Financial Info Opened Successfully


In [240]:
download(driver,directory)

Audit = True
Current amount of files in dir: 0
File number 0
Starting Download
First Download
New amount of files in dir: 1
Current amount of files in dir: 1
File number 1
Starting Download
New amount of files in dir: 1
No file was downloaded. Repeating iteration...
Nav. to financial failed
Fail number 1
Captcha Solved
Financial Info Opened Successfully
Audit = True
Current amount of files in dir: 1
File number 1
Starting Download
New amount of files in dir: 2
Current amount of files in dir: 2
File number 2
Captcha Solved
Starting Download
New amount of files in dir: 3
Current amount of files in dir: 3
File number 3
Starting Download
New amount of files in dir: 4
Current amount of files in dir: 4
File number 4
Captcha Solved
Starting Download
New amount of files in dir: 5
Current amount of files in dir: 5
File number 5
Captcha Solved
Starting Download
Load Failed...
Nav. to financial failed
Fail number 1
Captcha Solved
Captcha Solved
Financial Info Opened Successfully
Audit = True
Curr

StaleElementReferenceException: Message: stale element reference: element is not attached to the page document
  (Session info: chrome=112.0.5615.139)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0121DCE3+50899]
	(No symbol) [0x011AE111]
	(No symbol) [0x010B5588]
	(No symbol) [0x010B7EDB]
	(No symbol) [0x010B7DB0]
	(No symbol) [0x010B8690]
	(No symbol) [0x0110E98D]
	(No symbol) [0x010FB8FC]
	(No symbol) [0x0110E01C]
	(No symbol) [0x010FB6F6]
	(No symbol) [0x010D7708]
	(No symbol) [0x010D886D]
	GetHandleVerifier [0x01483EAE+2566302]
	GetHandleVerifier [0x014B92B1+2784417]
	GetHandleVerifier [0x014B327C+2759788]
	GetHandleVerifier [0x012B5740+672048]
	(No symbol) [0x011B8872]
	(No symbol) [0x011B41C8]
	(No symbol) [0x011B42AB]
	(No symbol) [0x011A71B7]
	(No symbol) [0x76AE0099]
	RtlGetAppContainerNamedObjectPath [0x772F7B6E+286]
	RtlGetAppContainerNamedObjectPath [0x772F7B3E+238]


In [30]:
navFin(driver)
pdfShow(driver)

Captcha Solved
Financial Info Opened Successfully
0
No audited financial statements found. Resorting to downloading the regular financials


#### Testing coordinates for download function

In [236]:
#Columna random
# y = driver.find_element(By.ID,"frmInformacionCompanias:tblInformacionAnual:0:j_idt589")
#PDF
# y = driver.find_element(By.ID,"frmInformacionCompanias:tblInformacionAnual:0:j_idt599")
# y.location
#Left corner of box
x = driver.find_element(By.ID,"dlgPresentarDocumentoPdf")
print(x.location)
x_s,y_s=x.size.values()
print(y_s)
x_s -= 28
y_s = y_s/2
x_s = x_s/2
print((x_s,y_s))

#0,0 Coordinate?
# x = driver.find_element(By.TAG_NAME,"body")
# x.location
# z = driver.find_element(By.CLASS_NAME,"ui-button-icon-left.ui-icon.ui-c.btn-cerrar")
# z.click()
# x = 0
# y = 0
# z.location

{'x': 28, 'y': 0}
932
(285.5, 466.0)


In [201]:
print(driver.get_window_size())
# driver.set_window_size(1004,708)

{'width': 1004, 'height': 708}


In [237]:
# for i in range(1):
action = ActionChains(driver)

#Download Button
###(When using Gaming PC)
# action.move_to_element_with_offset(x,870,70)
# action.click()
# action.perform()
###When using regular PC
# action.move_to_element_with_offset(x,360,-225)
# action.click()
# action.perform()
# #         X Button
# # time.sleep(4)
# action.move_to_element_with_offset(x,470,-280)
# action.click()
# action.perform()

#Download Button (After first download)
###When using Gaming PC
# action.move_to_element_with_offset(x,870,90)
# action.click()
# action.perform()
###When using regular PC (Reduce 70 for true regular PC. CURRENT IS FOR DHUB PC IN EDWIN'S USERNAME)
# action.move_to_element_with_offset(x,365,-205)
# action.click()
# action.perform()

#X Button (After first download)
###When Using Gaming PC
# action.move_to_element_with_offset(x,970,45)
# action.click()
# action.perform()

###When using regular PC
# action.move_to_element_with_offset(x,450,-245)
# action.click()
# action.perform()
#     action.send_keys(Keys.ENTER)
# action.click()
# action.perform()
#     action.send_keys(Keys.TAB*2)
#     action.send_keys(Keys.ENTER)
#     action.send_keys(Keys.ESCAPE)
#     action.perform()

In [ ]:
def checkDuplicates(driver):
    path = r"C:\Users\EDWIN\OneDrive - Universidad San Francisco de Quito\USFQ Online\Décimosegundo Semestre\Web Scraping Course\Financials"
    files = os.listdir(path)
    for file in files:
        check = "(" in file)
        if check == True:
            os.remove(f"{path}\\{file}")
            break
    return check

In [5]:
lookForAll(exps)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="frmInformacionCompanias:tblInformacionAnual:j_idt587:filter"]"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x01156643]
	(No symbol) [0x010EBE21]
	(No symbol) [0x00FEDA9D]
	(No symbol) [0x01021342]
	(No symbol) [0x0102147B]
	(No symbol) [0x01058DC2]
	(No symbol) [0x0103FDC4]
	(No symbol) [0x01056B09]
	(No symbol) [0x0103FB76]
	(No symbol) [0x010149C1]
	(No symbol) [0x01015E5D]
	GetHandleVerifier [0x013CA142+2497106]
	GetHandleVerifier [0x013F85D3+2686691]
	GetHandleVerifier [0x013FBB9C+2700460]
	GetHandleVerifier [0x01203B10+635936]
	(No symbol) [0x010F4A1F]
	(No symbol) [0x010FA418]
	(No symbol) [0x010FA505]
	(No symbol) [0x0110508B]
	BaseThreadInitThunk [0x753F00F9+25]
	RtlGetAppContainerNamedObjectPath [0x770D7BBE+286]
	RtlGetAppContainerNamedObjectPath [0x770D7B8E+238]


In [9]:
import time
start = time.time()
driver,datos = lookForAll(exps)
end = time.time()
print(f"Total iteration time = {end-start}")

,Expediente,Identificación,Nombre,Nacionalidad,Cargo,Fecha nombramiento,Periodo,Número registro mercantil,Fecha registro mercantil,RL o ADM,PDF
0,33283,101878130,PACHECO GARCIA ROMULO IVAN,ECUADOR,GERENTE GENERAL,2022-03-08,1,1458,2022-05-30,RL,NaN
1,33283,103194312,GORDILLO MONTALVAN MILTON GEOVANNI,ECUADOR,PRESIDENTE,2022-03-08,1,1457,2022-05-30,ADM,NaN
2,33283,104353701,LEON BACUILIMA JUAN CARLOS,ECUADOR,GERENTE GENERAL,2020-03-20,1,1521,2020-07-10,RL,NaN
3,33283,101878130,PACHECO GARCIA ROMULO IVAN,ECUADOR,GERENTE GENERAL,2021-03-05,1,886,2021-04-01,RL,NaN
4,33283,104353701,LEON BACUILIMA JUAN CARLOS,ECUADOR,GERENTE GENERAL,2019-03-19,1,784,2019-04-15,RL,NaN
...,...,...,...,...,...,...,...,...,...,...,...
175,33389,102285384,IÑIGUEZ SANCHEZ FELIPE JAVIER,ECUADOR,GERENTE GENERAL,2011-09-02,2,1833,2011-09-08,RL,NaN
176,33389,102285384,IÑIGUEZ SANCHEZ FELIPE JAVIER,ECUADOR,GERENTE GENERAL,2009-09-01,2,1478,2009-09-01,RL,NaN
177,33389,102285384,IÑIGUEZ SANCHEZ FELIPE JAVIER,ECUADOR,GERENTE GENERAL,2007-01-10,2,55,2007-01-10,RL,NaN
178,33389,102115904,IZQUIERDO ABAD SEBASTIAN,ECUADOR,GERENTE GENERAL,2008-12-17,2,162,2009-02-02,RL,NaN


363.6829149723053
